# 基本测试

In [3]:
import torch
import numpy as np
from datasets import Dataset as HFDataset
from datasets import ClassLabel, Features, Sequence, Value, concatenate_datasets, interleave_datasets
from torch.utils.data import DataLoader

/home/user/anaconda3/envs/LQ_signal/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# --- 1. 创建或加载单独的 Hugging Face Dataset 对象 ---
# 通常，你会从文件（CSV, JSON, Parquet等）或 Hugging Face Hub 加载数据。
# 为了示例，我们从 Python 字典创建一些虚拟数据。

# 定义所有数据集共享的特征结构
# 假设 'data' 是一个长度为10的时间序列，'label' 是分类标签
features = Features({
    'data': Sequence(Value(dtype='float32'), length=10), # 或者 length=-1 表示可变长度
    'label': ClassLabel(names=['normal', 'fault_type_1', 'fault_type_2']) # 示例标签
})

# 创建示例数据集字典
hf_datasets_dict = {
    'dataset_A_engine1': HFDataset.from_dict({
        'data': np.random.rand(100, 10).astype(np.float32), # 100个样本，每个样本10个特征点
        'label': np.random.randint(0, 3, 100) # 100个标签
    }, features=features),
    'dataset_B_engine2': HFDataset.from_dict({
        'data': np.random.rand(50, 10).astype(np.float32), # 50个样本
        'label': np.random.randint(0, 3, 50)
    }, features=features),
    'dataset_C_engine3': HFDataset.from_dict({
        'data': np.random.rand(75, 10).astype(np.float32), # 75个样本
        'label': np.random.randint(0, 3, 75)
    }, features=features),
}

print("原始数据集信息:")
for name, ds in hf_datasets_dict.items():
    print(f"- {name}: {len(ds)} 个样本, 特征: {ds.features}")

原始数据集信息:
- dataset_A_engine1: 100 个样本, 特征: {'data': Sequence(feature=Value(dtype='float32', id=None), length=10, id=None), 'label': ClassLabel(names=['normal', 'fault_type_1', 'fault_type_2'], id=None)}
- dataset_B_engine2: 50 个样本, 特征: {'data': Sequence(feature=Value(dtype='float32', id=None), length=10, id=None), 'label': ClassLabel(names=['normal', 'fault_type_1', 'fault_type_2'], id=None)}
- dataset_C_engine3: 75 个样本, 特征: {'data': Sequence(feature=Value(dtype='float32', id=None), length=10, id=None), 'label': ClassLabel(names=['normal', 'fault_type_1', 'fault_type_2'], id=None)}


In [6]:
# --- 2. 为每个数据集的样本添加 'dataset_id' ---
# 这在后续分析或需要知道样本来源时非常有用。
processed_hf_datasets_list = []
dataset_id_map = {} # 用于将整数ID映射回名称（如果需要）

for i, (ds_name, hf_ds) in enumerate(hf_datasets_dict.items()):
    dataset_id_map[i] = ds_name
    def add_dataset_id(example, idx, current_ds_id=i): # 使用默认参数捕获当前的ds_id
        example['dataset_id'] = current_ds_id # 添加整数ID
        # 你也可以直接添加 ds_name，但整数ID有时更方便处理
        # example['dataset_name'] = ds_name
        return example

    # .map() 是一个非常强大的函数，可以对数据集中的每个样本应用一个函数
    # with_indices=True 允许我们的函数接收样本的索引
    processed_ds = hf_ds.map(add_dataset_id, with_indices=True, batched=False)
    processed_hf_datasets_list.append(processed_ds)

print("\n添加 'dataset_id' 后的第一个数据集的第一个样本:")
print(processed_hf_datasets_list[0][0])

Map: 100%|██████████| 75/75 [00:00<00:00, 11478.66 examples/s]


添加 'dataset_id' 后的第一个数据集的第一个样本:
{'data': [0.23012354969978333, 0.5572117567062378, 0.1324310153722763, 0.8973038196563721, 0.0951453298330307, 0.678874671459198, 0.24881014227867126, 0.7016008496284485, 0.010452426970005035, 0.4540610909461975], 'label': 2, 'dataset_id': 0}


In [8]:
# --- 3. 用于 'val'/'test' 模式: 顺序合并数据集 ---
# `concatenate_datasets` 将按列表顺序将数据集首尾相连。
concatenated_dataset = concatenate_datasets(processed_hf_datasets_list)

print(f"\n合并后的数据集 (val/test模式):")
print(f"- 总长度: {len(concatenated_dataset)}")
print(f"- 特征: {concatenated_dataset.features}") # 特征应该包含新的 'dataset_id'
# 验证前几个和后几个样本的 dataset_id
print(f"- 第1个样本: {concatenated_dataset[0]}")
print(f"- 第100个样本 (原ds_A结束，ds_B开始处): {concatenated_dataset[100]}") # 假设ds_A有100个
print(f"- 最后一个样本: {concatenated_dataset[-1]}")


合并后的数据集 (val/test模式):
- 总长度: 225
- 特征: {'data': Sequence(feature=Value(dtype='float32', id=None), length=10, id=None), 'label': ClassLabel(names=['normal', 'fault_type_1', 'fault_type_2'], id=None), 'dataset_id': Value(dtype='int64', id=None)}
- 第1个样本: {'data': [0.23012354969978333, 0.5572117567062378, 0.1324310153722763, 0.8973038196563721, 0.0951453298330307, 0.678874671459198, 0.24881014227867126, 0.7016008496284485, 0.010452426970005035, 0.4540610909461975], 'label': 2, 'dataset_id': 0}
- 第100个样本 (原ds_A结束，ds_B开始处): {'data': [0.9916283488273621, 0.6911718845367432, 0.3121793866157532, 0.45244041085243225, 0.16339486837387085, 0.9700718522071838, 0.6193543076515198, 0.3230269253253937, 0.2161582112312317, 0.8906893134117126], 'label': 2, 'dataset_id': 1}
- 最后一个样本: {'data': [0.5071959495544434, 0.8304657936096191, 0.41147640347480774, 0.7734295725822449, 0.9347462058067322, 0.698542058467865, 0.25387296080589294, 0.6484841704368591, 0.017896374687552452, 0.6011180877685547], 'label':

In [9]:
# --- 4. 用于 'train' 模式: 交错/平衡采样数据集 ---
# `interleave_datasets` 允许你从多个数据集中采样，可以控制采样概率和停止策略。

# 4a. 等概率采样，当任一数据集耗尽时停止 (可能不是最佳平衡)
# interleaved_train_dataset_simple = interleave_datasets(
#     processed_hf_datasets_list,
#     stopping_strategy='first_exhausted' # 'first_exhausted' 或 'all_exhausted'
# )
# print(f"\n交错数据集 (train模式 - first_exhausted):")
# print(f"- 长度: {len(interleaved_train_dataset_simple)}") # 长度会是 min_len * num_datasets
# print(f"- 第1个样本: {interleaved_train_dataset_simple[0]}")

# 4b. 等概率采样，当所有数据集都至少遍历完一次长的那个时（通过重采样短的）
# `stopping_strategy='all_exhausted'` 会从较短的数据集中重复采样，直到最长的数据集耗尽。
# 这与你之前 `BalancedDataset` 中训练模式的逻辑相似，即小的被过采样。
num_train_datasets = len(processed_hf_datasets_list)
equal_probabilities = [1.0 / num_train_datasets] * num_train_datasets

interleaved_balanced_train_dataset = interleave_datasets(
    processed_hf_datasets_list,
    probabilities=equal_probabilities,
    stopping_strategy='all_exhausted', # 关键参数，实现对小数据集的过采样
    seed=42 # 为了可复现性
)
# 注意: `interleave_datasets` 返回的 `IterableDataset` 默认可能没有 `__len__`。
# 如果你需要一个固定的epoch长度，通常是在训练循环中控制迭代的步数，
# 或者你可以用 `.take(N)` 来创建一个固定大小的数据集视图（但这会创建一个新的IterableDataset）。
# 对于 PyTorch DataLoader，IterableDataset 不需要实现 `__len__`。
print(f"\n交错并平衡采样的数据集 (train模式 - all_exhausted):")
# print(f"- 长度: 通常 IterableDataset 没有固定长度，除非被包装或显式定义")
# 我们可以尝试迭代一些样本来看看效果
print("从平衡的训练数据集中获取前10个样本:")
for i, sample in enumerate(interleaved_balanced_train_dataset.take(15)): # .take(N) 获取N个样本
    print(f"  Sample {i}: data[0]={sample['data'][0]:.2f}, label={sample['label']}, dataset_id={sample['dataset_id']} ({dataset_id_map[sample['dataset_id']]})")
    if i >= 14:
        break

# 4c. 根据数据集大小调整采样概率 (可选，如果你不希望完全平衡，而是按比例)
# total_samples = sum(len(ds) for ds in processed_hf_datasets_list)
# size_based_probabilities = [len(ds) / total_samples for ds in processed_hf_datasets_list]
# interleaved_weighted_train_dataset = interleave_datasets(
#     processed_hf_datasets_list,
#     probabilities=size_based_probabilities,
#     stopping_strategy='all_exhausted',
#     seed=42
# )
# print(f"\n按数据集大小加权交错采样的数据集 (train模式 - all_exhausted):")
# for i, sample in enumerate(interleaved_weighted_train_dataset.take(10)):
#     print(f"  Sample {i}: dataset_id={sample['dataset_id']}")
#     if i >= 9:
#         break


交错并平衡采样的数据集 (train模式 - all_exhausted):
从平衡的训练数据集中获取前10个样本:
  Sample 0: data[0]=0.76, label=1, dataset_id=2 (dataset_C_engine3)
  Sample 1: data[0]=0.99, label=2, dataset_id=1 (dataset_B_engine2)
  Sample 2: data[0]=0.01, label=2, dataset_id=2 (dataset_C_engine3)
  Sample 3: data[0]=0.40, label=1, dataset_id=2 (dataset_C_engine3)
  Sample 4: data[0]=0.23, label=2, dataset_id=0 (dataset_A_engine1)
  Sample 5: data[0]=0.72, label=2, dataset_id=2 (dataset_C_engine3)
  Sample 6: data[0]=0.02, label=2, dataset_id=2 (dataset_C_engine3)
  Sample 7: data[0]=0.21, label=2, dataset_id=2 (dataset_C_engine3)
  Sample 8: data[0]=0.46, label=2, dataset_id=0 (dataset_A_engine1)
  Sample 9: data[0]=0.03, label=1, dataset_id=1 (dataset_B_engine2)
  Sample 10: data[0]=0.08, label=1, dataset_id=1 (dataset_B_engine2)
  Sample 11: data[0]=0.15, label=1, dataset_id=2 (dataset_C_engine3)
  Sample 12: data[0]=0.62, label=1, dataset_id=1 (dataset_B_engine2)
  Sample 13: data[0]=0.62, label=1, dataset_id=2 (dat

In [10]:
# --- 5. 设置数据集格式以便与 PyTorch 一起使用 ---
# 这会告诉数据集在被索引时返回 PyTorch 张量而不是 Python列表/Numpy数组。
concatenated_dataset.set_format(type='torch', columns=['data', 'label', 'dataset_id'])
# 对于 IterableDataset (interleave_datasets 的输出)，set_format 的行为略有不同，
# 它通常在数据流经它时应用转换。
# 如果直接将interleaved_balanced_train_dataset传递给PyTorch DataLoader，
# 并且其内部数据集已经知道如何输出正确类型（例如，如果它们来自 `Dataset.from_dict` 时 NumPy 数组可以被 PyTorch 直接处理），
# 或者在 `.map` 中转换为 PyTorch 张量，那么可能不需要显式的 `set_format`。
# 但通常最好还是加上，或者在 collate_fn 中确保类型正确。
# Hugging Face `datasets` 通常能很好地与 PyTorch DataLoader 配合。
# 为了安全起见，我们可以这样做：
interleaved_balanced_train_dataset = interleaved_balanced_train_dataset.with_format("torch")


# --- 6. 与 PyTorch DataLoader 一起使用 ---

# 验证/测试 DataLoader
val_dataloader_hf = DataLoader(
    concatenated_dataset,
    batch_size=32,
    shuffle=False # 验证/测试时通常不打乱
)

# 训练 DataLoader
# 注意：对于 IterableDataset，shuffle 参数在 DataLoader 中无效。
# shuffle 应该在 `interleave_datasets` (通过 seed 和内部缓冲) 或 Dataset 本身层面处理。
# `interleave_datasets` 内部有洗牌逻辑，可以通过 `buffer_size` 参数控制其洗牌效果。
train_dataloader_hf = DataLoader(
    interleaved_balanced_train_dataset,
    batch_size=32
    # num_workers > 0 在 IterableDataset 上的支持可能因 datasets 版本和具体情况而异，
    # 需要测试。对于简单的 IterableDataset，num_workers=0 通常最安全。
    # 对于复杂的预处理，你可能希望在 .map() 中进行，或者使用 .shard() 来手动分配给 worker。
)

print(f"\n使用 PyTorch DataLoader:")
print("从 val_dataloader_hf 获取一个批次:")
for batch in val_dataloader_hf:
    print(f"- 特征批次形状: {batch['data'].shape}")
    print(f"- 标签批次形状: {batch['label'].shape}")
    print(f"- Dataset ID 批次: {batch['dataset_id']}")
    print(f"- 第一个样本的 Dataset ID (来自map): {dataset_id_map[batch['dataset_id'][0].item()]}")
    break # 只看一个批次

print("\n从 train_dataloader_hf 获取一个批次:")
# 注意: 训练循环通常基于步数，而不是 epoch (因为 IterableDataset 可能没有固定长度)
num_train_steps = 10 # 假设我们训练10步
for step, batch in enumerate(train_dataloader_hf):
    if step >= num_train_steps and num_train_steps > 0 : # 限制迭代次数，因为IterableDataset可能是无限的
        break
    print(f"Step {step}:")
    print(f"- 特征批次形状: {batch['data'].shape}")
    print(f"- 标签批次形状: {batch['label'].shape}")
    print(f"- Dataset ID 批次: {batch['dataset_id']}")
    if step == 0: # 只打印第一个批次的ID映射
         print(f"- 第一个样本的 Dataset ID (来自map): {dataset_id_map[batch['dataset_id'][0].item()]}")
    if step >= 2: # 打印少量批次用于演示
        break 
print("...")


使用 PyTorch DataLoader:
从 val_dataloader_hf 获取一个批次:
- 特征批次形状: torch.Size([32, 10])
- 标签批次形状: torch.Size([32])
- Dataset ID 批次: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])
- 第一个样本的 Dataset ID (来自map): dataset_A_engine1

从 train_dataloader_hf 获取一个批次:
Step 0:
- 特征批次形状: torch.Size([32, 10])
- 标签批次形状: torch.Size([32])
- Dataset ID 批次: tensor([2, 1, 2, 2, 0, 2, 2, 2, 0, 1, 1, 2, 1, 2, 1, 0, 1, 0, 2, 1, 2, 1, 2, 2,
        2, 0, 1, 0, 0, 2, 2, 2])
- 第一个样本的 Dataset ID (来自map): dataset_C_engine3
Step 1:
- 特征批次形状: torch.Size([32, 10])
- 标签批次形状: torch.Size([32])
- Dataset ID 批次: tensor([0, 1, 1, 0, 0, 1, 0, 2, 1, 2, 2, 0, 2, 2, 1, 0, 2, 0, 0, 0, 2, 1, 2, 2,
        1, 1, 0, 0, 2, 1, 1, 2])
Step 2:
- 特征批次形状: torch.Size([32, 10])
- 标签批次形状: torch.Size([32])
- Dataset ID 批次: tensor([1, 1, 1, 0, 0, 1, 0, 1, 2, 0, 0, 0, 0, 1, 1, 2, 1, 1, 2, 0, 0, 0, 2, 1,
        0, 1, 0, 2, 1, 1, 0, 1])
...


# IdIncludedDataset

In [ ]:
# =====================================================================================
# 自定义包装类 IdIncludedDataset
# =====================================================================================
class IdIncludedDataset(Dataset):
    def __init__(self, dataset_dict):
        """
        包装一个 PyTorch Dataset 字典，使得每个样本都包含其原始ID。

        Args:
            dataset_dict (dict): 一个字典，键是字符串ID，值是 PyTorch Dataset 对象。
                                 例如：{'id1': train_dataset_for_id1, 'id2': train_dataset_for_id2}
                                 其中 train_dataset_for_id1 等实例的 __getitem__ 返回 (x, y)。
        """
        self.dataset_dict_refs = dataset_dict # 保存对原始数据集字典的引用
        self.flat_sample_map = [] # 用于全局索引到 (id, 原始数据集中的索引) 的映射

        for id_str, original_dataset in self.dataset_dict_refs.items():
            if original_dataset is None:
                print(f"警告: ID '{id_str}' 对应的 dataset 为 None，已跳过。")
                continue
            if len(original_dataset) == 0:
                print(f"警告: ID '{id_str}' 对应的 dataset 为空，已跳过。")
                continue
            
            for i in range(len(original_dataset)):
                self.flat_sample_map.append({'id': id_str, 'original_idx': i})
        
        self._total_samples = len(self.flat_sample_map)

    def __len__(self):
        """
        返回所有原始数据集中样本的总数。
        """
        return self._total_samples

    def __getitem__(self, global_idx):
        """
        根据全局索引获取样本，并返回 (id, (x, y))。

        Args:
            global_idx (int): 全局样本索引。

        Returns:
            tuple: (str, tuple), 即 (id, (x, y))
                   其中 x 是特征数据, y 是标签。
        """
        if global_idx < 0 or global_idx >= self._total_samples:
            raise IndexError(f"全局索引 {global_idx} 超出范围 (总样本数: {self._total_samples})")

        sample_info = self.flat_sample_map[global_idx]
        original_id = sample_info['id']
        idx_in_original_dataset = sample_info['original_idx']

        # 从原始数据集中获取 (x, y)
        original_dataset_instance = self.dataset_dict_refs[original_id]
        x, y = original_dataset_instance[idx_in_original_dataset]
        
        return  (x, y),original_id


# dataset 包装器 + sampler

In [27]:
import torch
from torch.utils.data import Dataset, DataLoader,Sampler
import random

In [28]:
class Default_dataset(Dataset): # THU_006or018_basic
    def __init__(self, data, metadata, args_data, args_task, mode="train"):
        """
        简化的数据集类
        Args:
            data: 输入数据，可能是单一ID的数据 [L, C]，或字典格式 {ID: 数据}
            metadata: 数据元信息，格式为 {ID: {字段: 值}} 字典
            args_data: 数据处理参数
            args_task: 任务参数
            mode: 数据模式，可选 "train", "valid", "test"
        """
        self.key = list(data.keys())[0]
        self.data = data[self.key]  # 取出第一个键的数据
        self.metadata = metadata
        self.args_data = args_data
        self.mode = mode
        
        # 数据处理参数
        self.window_size = args_data.window_size
        self.stride = args_data.stride
        self.train_ratio = args_data.train_ratio
        
        # 数据预处理
        self.processed_data = []  # 存储处理后的样本
                
        # 处理数据
        self.prepare_data()
        
    def prepare_data(self):
        """
        准备数据：将原始数据按窗口大小和步长分割成样本
        如果mode是train或valid，则划分数据集
        """
        self._process_single_data(self.data)

        # 如果是train或valid模式，进行数据集划分
        if self.mode in ["train", "valid"]:
            self._split_data_for_mode()
            
        self.total_samples = len(self.processed_data) # L'
        self.label = self.metadata[self.key]["Label"]
    
    def _process_single_data(self, sample_data):
        """
        处理单个数据样本，应用滑动窗口
        """
        data_length = len(sample_data)
        num_samples = max(0, (data_length - self.window_size) // self.stride + 1)
        
        for i in range(num_samples):
            start_idx = i * self.stride
            end_idx = start_idx + self.window_size
            
            self.processed_data.append(sample_data[start_idx:end_idx])

    def _split_data_for_mode(self):
        """
        根据当前模式划分数据集
        """
        if not self.processed_data:
            return
            
        # 计算划分点
        total_samples = len(self.processed_data)
        train_size = int(self.train_ratio * total_samples)
        
        if self.mode == "train":
            # 训练模式只保留训练数据
            self.processed_data = self.processed_data[:train_size]
        elif self.mode == "valid":
            # 验证模式只保留验证数据
            self.processed_data = self.processed_data[train_size:]
        self.total_samples = len(self.processed_data)
    
    def __len__(self):
        """返回数据集长度"""
        return self.total_samples
    
    def __getitem__(self, idx):
        """获取指定索引的样本"""
        if idx >= self.total_samples:
            raise IndexError(f"索引 {idx} 超出范围")
        
        sample = self.processed_data[idx]

        
        return sample, self.label

## sampler

In [31]:
class BalancedIdSampler(Sampler):
    def __init__(self, data_source: IdIncludedDataset, common_samples_per_id=None, shuffle_within_id=True, shuffle_all=True):
        """
        Sampler 实现对不同原始数据集(ID)的平衡加载。

        Args:
            data_source (IdIncludedDataset): 被包装的数据集，必须是 IdIncludedDataset 类型。
            common_samples_per_id (int, optional): 
                每个ID在一个epoch中被采样的目标次数。
                如果为 None, 则默认为样本量最大的ID的样本数 (即对小ID进行过采样)。
            shuffle_within_id (bool): 是否在为每个ID选择样本时进行随机选择。
            shuffle_all (bool): 是否在最后将所有选出的索引进行全局随机打乱。
        """
        super().__init__(data_source)
        self.data_source = data_source
        self.shuffle_within_id = shuffle_within_id
        self.shuffle_all = shuffle_all

        # 1. 按ID对全局索引进行分组
        self.indices_per_id = {}
        if not hasattr(self.data_source, 'flat_sample_map'):
            raise ValueError("data_source 必须是 IdIncludedDataset 的实例，或具有 'flat_sample_map' 属性。")

        for global_idx, sample_info in enumerate(self.data_source.flat_sample_map):
            original_id = sample_info['id']
            if original_id not in self.indices_per_id:
                self.indices_per_id[original_id] = []
            self.indices_per_id[original_id].append(global_idx)

        self.id_list = list(self.indices_per_id.keys())
        if not self.id_list: # 如果没有任何有效的ID
            self._num_samples_epoch = 0
            self.target_samples_per_id = 0
            return

        # 2. 确定每个ID的目标采样数
        num_actual_samples_per_id = {id_str: len(indices) for id_str, indices in self.indices_per_id.items()}
        
        if common_samples_per_id is None:
            # 默认目标是最大ID的样本量
            if not num_actual_samples_per_id:
                 self.target_samples_per_id = 0 # 防止空字典
            else:
                 self.target_samples_per_id = max(num_actual_samples_per_id.values()) if num_actual_samples_per_id else 0
        else:
            self.target_samples_per_id = common_samples_per_id
        
        # 3. 计算一个epoch的总样本数
        self._num_samples_epoch = self.target_samples_per_id * len(self.id_list)

    def __iter__(self):
        if not self.id_list or self.target_samples_per_id == 0:
            return iter([])

        all_epoch_indices = []
        for id_str in self.id_list:
            id_specific_global_indices = self.indices_per_id[id_str]
            num_actual_id_samples = len(id_specific_global_indices)

            if num_actual_id_samples == 0:
                continue # 这个ID没有样本

            if num_actual_id_samples < self.target_samples_per_id:
                # 过采样: 需要重复采样
                # random.choices 实现带放回采样
                chosen_for_id = random.choices(id_specific_global_indices, k=self.target_samples_per_id)
            elif num_actual_id_samples > self.target_samples_per_id:
                # 欠采样或精确采样: 不带放回采样
                if self.shuffle_within_id:
                    chosen_for_id = random.sample(id_specific_global_indices, k=self.target_samples_per_id)
                else: # 按顺序取前k个
                    chosen_for_id = id_specific_global_indices[:self.target_samples_per_id]
            else: # num_actual_id_samples == self.target_samples_per_id
                # 样本数正好等于目标数
                if self.shuffle_within_id:
                    chosen_for_id = random.sample(id_specific_global_indices, k=num_actual_id_samples)
                else:
                    chosen_for_id = list(id_specific_global_indices) # 复制列表
            
            all_epoch_indices.extend(chosen_for_id)

        if self.shuffle_all:
            random.shuffle(all_epoch_indices) # 打乱所有ID组合后的总索引列表
        
        return iter(all_epoch_indices)

    def __len__(self):
        return self._num_samples_epoch

## test

In [30]:
# --- 模拟参数和数据 ---
class ArgsSim:
    def get(self, key, default):
        return getattr(self, key, default)

args_data_sim = ArgsSim()
args_data_sim.window_size = 5
args_data_sim.train_ratio = 0.8
args_data_sim.stride = 5

args_task_sim = ArgsSim() # dummy

# 模拟不同大小的原始数据集
dataset_dict_sim = {
    'id_A': Default_dataset(
        data={'id_A': np.random.rand(20, 3)}, # 初始20个“时间点” -> 4个原始样本 -> 3个训练样本
        metadata={'id_A': {'Label': 0}},
        args_data=args_data_sim, args_task=args_task_sim, mode="train"
    ),
    'id_B': Default_dataset(
        data={'id_B': np.random.rand(50, 3)}, # 初始50个“时间点” -> 10个原始样本 -> 8个训练样本
        metadata={'id_B': {'Label': 1}},
        args_data=args_data_sim, args_task=args_task_sim, mode="train"
    ),
    'id_C': Default_dataset( # 一个可能为空的数据集或处理后样本很少
        data={'id_C': np.random.rand(8, 3)}, # 初始8个“时间点” -> 1个原始样本 -> 0个训练样本 (如果train_ratio后为0)
                                             # 我们在 Default_dataset 中添加了确保至少1个样本的逻辑（如果可能）
        metadata={'id_C': {'Label': 2}},      # 假设处理后 id_C 有 1 个训练样本
        args_data=args_data_sim, args_task=args_task_sim, mode="train"
    )
}
# 预期 id_A 样本数: int(0.8 * (20//5)) = 3
# 预期 id_B 样本数: int(0.8 * (50//5)) = 8
# 预期 id_C 样本数: int(0.8 * (8//5)) = int(0.8*1) = 0, 但Default_dataset简化版可能产生1个 (如果原始数据足够)
# 假设 Default_dataset 简化版处理后：id_A: 3, id_B: 8, id_C: 1

# 包装数据集
wrapped_train_dataset = IdIncludedDataset(dataset_dict_sim)
print(f"IdIncludedDataset 总样本数: {len(wrapped_train_dataset)}") # 应该为 3 + 8 + 1 = 12

# --- 使用 BalancedIdSampler ---

# 策略1: 过采样小的ID，使得所有ID的样本量都与最大的ID相同 (id_B 有8个样本)
# target_samples_per_id 会是 8。epoch 总样本数 = 8 * 3 = 24
sampler_oversample = BalancedIdSampler(wrapped_train_dataset, common_samples_per_id=None) 
print(f"BalancedIdSampler (过采样至最大) epoch 长度: {len(sampler_oversample)}")

# 策略2: 为每个ID指定一个共同的采样数，比如每个ID采样5次
# epoch 总样本数 = 5 * 3 = 15
sampler_common_count = BalancedIdSampler(wrapped_train_dataset, common_samples_per_id=5)
print(f"BalancedIdSampler (每个ID采样5次) epoch 长度: {len(sampler_common_count)}")


# 创建 DataLoader
print("\n使用 sampler_oversample (过采样至最大ID的样本数):")
balanced_dataloader_1 = DataLoader(
    wrapped_train_dataset,
    batch_size=4, # 例如 batch_size 为 4
    sampler=sampler_oversample,
    # collate_fn=... # 如果需要特殊处理批次结构
)

# 迭代 DataLoader
id_counts_in_epoch_1 = {}
if len(balanced_dataloader_1) > 0:
    for batch_idx, data_batch in enumerate(balanced_dataloader_1):
        ids_in_batch, (xs_in_batch, ys_in_batch) = data_batch
        for id_val in ids_in_batch:
            id_counts_in_epoch_1[id_val] = id_counts_in_epoch_1.get(id_val, 0) + 1
        if batch_idx < 2: # 打印前2个批次的信息
             print(f"  Batch {batch_idx}: IDs: {ids_in_batch}, X shape: {xs_in_batch.shape}, Y shape: {ys_in_batch.shape}")
else:
    print("  DataLoader 为空 (可能是因为 wrapped_train_dataset 为空)")

print(f"  一个 epoch 中各ID的采样次数 (sampler_oversample): {id_counts_in_epoch_1}")
# 预期 id_A, id_B, id_C 的计数都接近或等于8

print("\n使用 sampler_common_count (每个ID采样5次):")
balanced_dataloader_2 = DataLoader(
    wrapped_train_dataset,
    batch_size=4,
    sampler=sampler_common_count
)
id_counts_in_epoch_2 = {}
if len(balanced_dataloader_2) > 0:
    for batch_idx, data_batch in enumerate(balanced_dataloader_2):
        ids_in_batch, (xs_in_batch, ys_in_batch) = data_batch
        for id_val in ids_in_batch:
            id_counts_in_epoch_2[id_val] = id_counts_in_epoch_2.get(id_val, 0) + 1
else:
    print("  DataLoader 为空")
print(f"  一个 epoch 中各ID的采样次数 (sampler_common_count): {id_counts_in_epoch_2}")
# 预期 id_A, id_B, id_C 的计数都接近或等于5

警告: ID 'id_C' 对应的 dataset 为空，已跳过。
IdIncludedDataset 总样本数: 11
BalancedIdSampler (过采样至最大) epoch 长度: 16
BalancedIdSampler (每个ID采样5次) epoch 长度: 10

使用 sampler_oversample (过采样至最大ID的样本数):
  Batch 0: IDs: ('id_A', 'id_B', 'id_A', 'id_B'), X shape: torch.Size([4, 5, 3]), Y shape: torch.Size([4])
  Batch 1: IDs: ('id_B', 'id_B', 'id_A', 'id_B'), X shape: torch.Size([4, 5, 3]), Y shape: torch.Size([4])
  一个 epoch 中各ID的采样次数 (sampler_oversample): {'id_A': 8, 'id_B': 8}

使用 sampler_common_count (每个ID采样5次):
  一个 epoch 中各ID的采样次数 (sampler_common_count): {'id_A': 5, 'id_B': 5}


/home/user/anaconda3/envs/LQ_signal/lib/python3.10/site-packages/torch/utils/data/sampler.py:64: UserWarning: `data_source` argument is not used and will be removed in 2.2.0.You may still have custom implementation that utilizes it.
  warnings.warn("`data_source` argument is not used and will be removed in 2.2.0."
